In [23]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from openpyxl import load_workbook
import time
import json
import pandas as pd
import random
import matplotlib.pyplot as plt

In [24]:
options = webdriver.ChromeOptions() # declare a Chrome Driver
options.add_experimental_option('prefs', {'profile.managed_default_content_settings.images': 2}) # set not to display pictures
browser = webdriver.Chrome(options= options)

In [25]:
url = 'https://www.jd.com/'

In [26]:
def startScraper(keywords):
    data_list = [] # declare a list to store dicts
    browser.get(url) # request url
    browser.find_element_by_id('key').send_keys(keywords) # enter keywords
    browser.find_element_by_id('key').send_keys(Keys.ENTER) # start search
    WebDriverWait(browser, 1000).until(
        EC.presence_of_all_elements_located(
            (By.CLASS_NAME, 'pn-next')
        )
    ) # wait till all the elements all loaded
    all_page = eval(browser.find_element_by_css_selector('span.p-skip em b').text) # get #num of pages
    count = 0 # set a counter
    # loop before the last page
    while True:
        try:
            count += 1
            # wait till all the information loaded
            WebDriverWait(browser, 1000).until(
                EC.presence_of_all_elements_located(
                    (By.CLASS_NAME, 'gl-item')
                )
            )
            browser.execute_script('document.documentElement.scrollTop=10000') # draw the bar to the bottom to load all the information
            time.sleep(random.randint(1, 3)) # randomly stop for seconds
            browser.execute_script('document.documentElement.scrollTop=0') # back to the top
            
            # extract information from label li
            lis = browser.find_elements_by_class_name('gl-item')
            for li in lis:
                # name
                name = li.find_element_by_xpath('.//div[@class="p-name p-name-type-2"]//em').text
                # price
                price = li.find_element_by_xpath('.//div[@class="p-price"]//i').text
                # #num of comments
                comment = li.find_elements_by_xpath('.//div[@class="p-commit"]//a')
                if comment:
                    comment = comment[0].text
                else:
                    comment = None
                # shop name
                shop_name = li.find_elements_by_class_name('J_im_icon')
                if shop_name:
                    shop_name = shop_name[0].text
                else:
                    shop_name = None
                # shop type
                shop_type = li.find_elements_by_class_name('goods-icons')
                if shop_type:
                    shop_type = shop_type[0].text
                else:
                    shop_type = None
                    
                # declare a dict to store data
                data_dict = {}
                data_dict['name'] = name
                data_dict['price'] = price
                data_dict['comment'] = comment
                data_dict['shop_name'] = shop_name
                data_dict['shop_type'] = shop_type
                
                data_list.append(data_dict)
                #print(data_dict)
        except Exception as e:
            continue
        if count == all_page:
            break
        # find element for next page and click
        fp_next = browser.find_element_by_css_selector('a.fp-next')
        browser.execute_script('document.documentElement.scrollTop = 10000')
        fp_next.click()
    return data_list

In [27]:
# prune the 'comments' to 'sales'
def prune_sales(comments):
    if comments == None:
        return int(0)
    elif comments == '':
        return int(0)
    elif comments[-1] != '+':
        return int(comments)
    elif comments[-2] == '万':
        return int(float(comments[:-2]) * 1e4)
    else:
        return int(comments[:-1])

In [28]:
# save data in excel
def main(keywords):
    result = startScraper(keywords)
    result_df = pd.DataFrame(result)
    result_df['sales'] = result_df['comment'].apply(prune_sales)
    with pd.ExcelWriter('sku_sells.xlsx', engine = 'openpyxl') as writer:
        writer.book = load_workbook('sku_sells.xlsx')
        result_df.to_excel(writer, sheet_name = keywords)

In [33]:
keywords = "雅诗兰黛" # enter keywords
main(keywords)